In [73]:
import pandas as pd

df = pd.read_csv(r'C:\Users\lesle\Desktop\lambda\Unit 4 Sprint 1\DS-Unit-4-Sprint-1-NLP\module3-document-classification\data\train.csv')

In [74]:
import texthero as hero

In [14]:
df.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [75]:
df['text'] = hero.clean(df['description'])

In [24]:
df['text']

0       sometimes whisky batched leftover barrels retu...
1       uncommon exclusive bottling year old cask stre...
2       release port version amrut' intermediate sherr...
3       year old single cask aged sherry butt interact...
4       quite herbal nose aromas dried tarragon parsle...
                              ...                        
4082    lies beneath surface dewar' blend finished vir...
4083    years maturation bourbon casks spent several m...
4084    bright delicate approachable showstopper deliv...
4085    ' calling pitmaster' dram nose muscular peat s...
4086    spicy sultanas greengage plums toffee new leat...
Name: text, Length: 4087, dtype: object

NameError: name 'spacy' is not defined

In [5]:
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer


nlp = spacy.load("en_core_web_lg")

In [6]:
# Wrap it all in a function
def get_lemmas(text):

    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.text.lower() not in nlp.Defaults.stop_words) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

In [74]:
tokens = []

for doc in tokenizer.pipe(df['reviews.text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc: 
        if token.text.lower() not in nlp.Defaults.stop_words:
            doc_tokens.append(token.text.lower())
   
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

NameError: name 'tokenizer' is not defined

In [7]:
df['lemmas'] = df['text'].apply(get_lemmas)

In [8]:
df['text2'] = hero.clean(df['lemmas'])

In [9]:
df['text2']

0       whisky batch leftover barrel return warehouse ...
1       uncommon exclusive bottling year old cask stre...
2       release port version amrut intermediate sherry...
3       year old single cask age sherry butt interact ...
4       herbal nose aroma dry tarragon parsley dill ch...
                              ...                        
4082    lie beneath surface dewar blend finish virgin ...
4083    year maturation bourbon cask spend month chard...
4084    bright delicate approachable showstopper deliv...
4085    call pitmaster dram nose muscular peat smoke p...
4086    spicy sultanas greengage plum toffee new leath...
Name: text2, Length: 4087, dtype: object

In [76]:
df['tfidf'] = (
    hero.tfidf(df['text'], max_features=100)
)
df[["tfidf", 'ratingCategory']].head(2)                       

,tfidf,ratingCategory
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1,"[0.14938057122315693, 0.0, 0.0, 0.0, 0.2347699...",0


In [78]:
df['pca'] = hero.pca(df['tfidf'])
hero.scatterplot(
    df, 
    col='pca', 
    color='ratingCategory', 
    title="PCA BBC Sport news"
)

In [16]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from xgboost import XGBClassifier


In [33]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)



vect = TfidfVectorizer(stop_words='english')

xgb = XGBClassifier()


lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', xgb)])

In [5]:
import xgboost as xgb




In [107]:
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', xgb)])

In [14]:
xgb.get_params().keys()

dict_keys(['objective', 'base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'gpu_id', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])

In [ ]:
{'lsi__vect__ngram_range': (1, 2),
 'lsi__vect__min_df': 0.03,
 'lsi__vect__max_features': 250,
 'lsi__vect__max_df': 0.7,
 'lsi__svd__n_components': 75,
 'clf__subsample': 0.8,
 'clf__n_estimators': 300,
 'clf__min_child_weight': 10,
 'clf__max_depth': 3,
 'clf__gamma': 5,
 'clf__colsample_bytree': 0.6}

In [48]:
parameters = {
    'lsi__svd__n_components': [50, 65, 75, 85],
    'lsi__vect__max_df':[.1, .2, .3, .4, .5, .6, .7],
    'lsi__vect__min_df': (.02, .25, .03, .035, .05),
    'lsi__vect__ngram_range': ((1,1), (1,2)), 
    'lsi__vect__max_features': (50, 100, 150, 200, 250, 500, 1000),
    'clf__n_estimators': (0, 5, 25, 50, 100, 250, 300, 350, 500),
    'clf__min_child_weight': [10, 15, 20],
    'clf__gamma': [0.5, 1, 1.5, 2, 5, 6, 7, 8, 9, 10],
    'clf__subsample': [0.6,.07, 0.8, .09, 1.0],
    'clf__colsample_bytree': [.04, 0.5, 0.6, 0.7],
    'clf__max_depth': [None, 2, 3, 4, 5, 10]
    
        
    
}

grid_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(df['text2'], df['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   24.0s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [51]:
grid_search.best_score_

0.7262050617531489

In [50]:
grid_search.best_params_

{'lsi__vect__ngram_range': (1, 1),
 'lsi__vect__min_df': 0.02,
 'lsi__vect__max_features': 1000,
 'lsi__vect__max_df': 0.5,
 'lsi__svd__n_components': 65,
 'clf__subsample': 0.6,
 'clf__n_estimators': 300,
 'clf__min_child_weight': 20,
 'clf__max_depth': None,
 'clf__gamma': 10,
 'clf__colsample_bytree': 0.6}

In [96]:
grid_search.best_params_

{'lsi__vect__ngram_range': (1, 1),
 'lsi__vect__min_df': 0.03,
 'lsi__vect__max_features': 500,
 'lsi__vect__max_df': 0.9,
 'lsi__svd__n_components': 100,
 'clf__n_estimators': 100,
 'clf__min_weight_fraction_leaf': 0,
 'clf__min_samples_split': 3,
 'clf__min_samples_leaf': 5,
 'clf__max_depth': 5}

In [65]:
vect = TfidfVectorizer(stop_words='english')

rfc = RandomForestClassifier()

In [102]:
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [47]:
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

In [28]:
parameters = {
 'lsi__vect__ngram_range': (1, 1),
 'lsi__vect__min_df': (0.03),
 'lsi__vect__max_features': (500),
 'lsi__vect__max_df': (0.9),
 'lsi__svd__n_components': (100),
 'clf__n_estimators': (100),
 'clf__min_weight_fraction_leaf': (0),
 'clf__min_samples_split': (3),
 'clf__min_samples_leaf': (5),
 'clf__max_depth': (5)
}

grid_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(df['text2'], df['ratingCategory'])

TypeError: Parameter value is not iterable or distribution (key='lsi__vect__min_df', value=0.03)

In [94]:
grid_search.best_score_

0.7076102863059737

In [54]:

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [55]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [64]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

NameError: name 'rfc' is not defined

In [57]:
grid_search = GridSearchCV(pipe,params, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(df['text'], df['ratingCategory'])

Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  4.5min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [26]:
grid_search.predict(['shit'])

array([1], dtype=int64)

In [58]:
grid_search.best_score_

0.723269303899622

In [28]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [29]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [30]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [32]:
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(df['text'], df['ratingCategory'])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 17.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [33]:
grid_search.best_score_

0.7279219399496638

In [34]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [41]:

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [42]:
X = get_word_vectors(df['text'])

len(X) == len(df['text'])

True

In [43]:
rfc.fit(X, df['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
rfc.score(X, df['ratingCategory'])

1.0

In [52]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [53]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [54]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [55]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [56]:
subNumber = 0

In [57]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

In [58]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [18]:
doc = nlp("Two bananas in pyjamas")

In [19]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [59]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [61]:
X = get_word_vectors(df['text2'])

len(X) == len(df['text2'])

True

In [68]:
X_test = get_word_vectors(test['description'])

In [66]:
rfc.fit(X, df['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [67]:
rfc.score(X, df['text2'])

0.0

In [69]:
rfc.predict(X_test)

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [70]:
test['ratingCategory'] = rfc.predict(X_test)

In [71]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

In [26]:
import spacy

nlp = spacy.load("en_core_web_lg")

def grab_adjs(text):
    doc = nlp(text)
    adjs = []
    
    for token in doc:
        print
        if token.pos == "ADJ":
            adjs.append(token.text)
            
    return adjs